In [84]:
from sympy import *
from IPython.display import display, Math

In [68]:
#init_printing()

Resulting light = light coming from pixel * transmittance + integral over all points inbetween of (light coming from that point * transmittance from that point to camera)

Cryteks "exponential height fog" is used to simply scale the constants $\sigma_{a,c}$ and $\sigma_{s,c}$ depending on the height:

(Reformulated to divide by c instead. That way increasing c increases fade (more intuitive))

In [69]:
c, sigma_ac, sigma_sc = symbols('c sigma_ac sigma_sc')

In [70]:
def sigma_a(y):
    return sigma_ac*exp(-y/c)
def sigma_s(y):
    return sigma_sc*exp(-y/c)

In [71]:
def sigma_t(y):
    return sigma_a(y) + sigma_s(y)

Transmittance from pixel to camera:\
camera: $o$, pixel: $o+D*d$\
(in theory $o, d$ are vector quantities, but only the y component matters. so theyre treated as scalars in the following)

In [72]:
o, t, d, D = symbols('o t d D',infinite=False)

In [73]:
i = Integral(sigma_t(o+t*d), (t, 0, D))

In [74]:
exp(-i)

exp(-Integral(sigma_ac*exp((-d*t - o)/c) + sigma_sc*exp((-d*t - o)/c), (t, 0, D)))

Solving the integral

Simple in the case that d = 0

In [80]:
i0 = Integral(sigma_t(o), (t, 0, D))
i0

Integral(sigma_ac*exp(-o/c) + sigma_sc*exp(-o/c), (t, 0, D))

In [81]:
i0 = sigma_t(o)*Integral(1, (t, 0, D))
i0

(sigma_ac*exp(-o/c) + sigma_sc*exp(-o/c))*Integral(1, (t, 0, D))

In [76]:
simplify(i0)

D*(sigma_ac + sigma_sc)*exp(-o/c)

otherwise

In [77]:
i

Integral(sigma_ac*exp((-d*t - o)/c) + sigma_sc*exp((-d*t - o)/c), (t, 0, D))

In [78]:
i.doit()

Piecewise(((-c*sigma_ac - c*sigma_sc)*exp((-D*d - o)/c)/d - (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d, (d > -oo) & (d < oo) & Ne(d, 0)), (D*(sigma_ac + sigma_sc), True))

In [105]:
display(Math('= \\frac{-(c*\\sigma_a + c*\\sigma_s)}{d} (e^{\\frac{-Dd-o}{c}} - e^{\\frac{-o}{c}})'))
display(Math('= \\frac{-(c*\\sigma_a + c*\\sigma_s)}{d} e^{\\frac{-o}{c}} (e^{\\frac{-Dd}{c}} - 1)'))
display(Math('= (c*\\sigma_a + c*\\sigma_s) e^{\\frac{-o}{c}} \\frac{(1 - e^{\\frac{-Dd}{c}})}{d}'))
display(Math('= (\\sigma_a + \\sigma_s) e^{\\frac{-o}{c}}c \\frac{(1 - e^{\\frac{-Dd}{c}})}{d}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

This form is better since the first part needs to be evaluated anyways. Even if d=0. Ie: less branching is required when evaluating this in a shader\
(result if d=0):

In [104]:
simplify(i0)

D*(sigma_ac + sigma_sc)*exp(-o/c)

Evaluating the limit of the second part (can also be seen when graphed) shows that this in fact does converge to the same result for d=0

In [117]:
l = Limit(c*(1-exp(-D*d/c))/d, d, 0)
l

Limit(c*(1 - exp(-D*d/c))/d, d, 0)

In [118]:
l.doit()

D

Then, the light from each point between the pixel and the camera has to be accumulated

In [60]:
l_i, t_2 = symbols('l_i t_2')

In [61]:
#Helper function to calculate transmittance from the origin along a direction up until a given depth
def transmittance(o, d, D):
    return exp(-Integral(sigma_t(o+t_2*d), (t_2, 0, D)))

In [62]:
def L_s(y): #in scatterd light at height
    return sigma_s(y)*l_i #just assume constant inscattering with a constanst phase function for this basic fog type

In [63]:
i = Integral(transmittance(o, d, t) * L_s(o+t*d), (t, 0, D))
i

Integral(l_i*sigma_sc*exp((-d*t - o)/c)*exp(-Integral(sigma_ac*exp((-d*t_2 - o)/c) + sigma_sc*exp((-d*t_2 - o)/c), (t_2, 0, t))), (t, 0, D))

In [64]:
i = i.doit()
i

Piecewise((-l_i*sigma_sc*exp(-(-c*sigma_ac - c*sigma_sc)*exp((-D*d - o)/c)/d + (-c*sigma_ac - c*sigma_sc)*exp(-o/c)/d)/(sigma_ac + sigma_sc) + l_i*sigma_sc/(sigma_ac + sigma_sc), ((d > -oo) & (d < oo) & Ne(d, 0) & Ne(sigma_ac + sigma_sc, 0)) | ((d > -oo) & (d < oo) & Ne(d, 0) & Ne(sigma_ac + sigma_sc, 0) & Ne(c, -d/(sigma_ac + sigma_sc)))), (-c*l_i*sigma_sc*exp((-D*d - o)/c)/d + c*l_i*sigma_sc*exp(-o/c)/d, ((d > -oo) & (d < oo) & Ne(d, 0)) | ((d > -oo) & (d < oo) & Ne(d, 0) & Ne(c, -d/(sigma_ac + sigma_sc)))), (-c*l_i*sigma_sc*exp(-o/c)/(c*sigma_ac*exp(D*sigma_ac)*exp(D*sigma_sc)*exp(D*d/c) + c*sigma_sc*exp(D*sigma_ac)*exp(D*sigma_sc)*exp(D*d/c) + d*exp(D*sigma_ac)*exp(D*sigma_sc)*exp(D*d/c)) + c*l_i*sigma_sc*exp(-o/c)/(c*sigma_ac + c*sigma_sc + d), Ne(c, -d/(sigma_ac + sigma_sc)) | ((d > -oo) & Ne(c, -d/(sigma_ac + sigma_sc))) | ((d < oo) & Ne(c, -d/(sigma_ac + sigma_sc))) | (Ne(d, 0) & Ne(c, -d/(sigma_ac + sigma_sc))) | (Ne(sigma_ac + sigma_sc, 0) & Ne(c, -d/(sigma_ac + sigma_sc))) |

In [65]:
i2 = i.args[0][0]

In [66]:
simplify(i2)

l_i*sigma_sc*(1 - exp(-c*(1 - exp(-D*d/c))*(sigma_ac + sigma_sc)*exp(-o/c)/d))/(sigma_ac + sigma_sc)